In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, QuantileTransformer
from sklearn.ensemble import BaggingRegressor, StackingRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import RFE
import lzma
import cloudpickle
import os
import category_encoders as ce
from joblib import Memory

In [2]:
df = pd.read_pickle('/tmp/all.pkl')
df_pos = pd.read_pickle('/tmp/pos_all.pkl')

def add_pos(df, df_pos):
    df_orig = df
    df = df.copy()
    df_pos = df_pos.copy()
    
    df_pos['city2'] = df_pos['city2_choume'].str.replace('[一二三四五六七八九十0-9]+丁目', '', regex=True)

    df_pos_pref = pd.concat([
        df_pos.groupby('prefecture')['latitude'].mean().rename('prefecture_latitude'),
        df_pos.groupby('prefecture')['longitude'].mean().rename('prefecture_longitude'),
        df_pos.groupby('prefecture')['latitude'].std().rename('prefecture_latitude_std'),
        df_pos.groupby('prefecture')['longitude'].std().rename('prefecture_longitude_std'),
        df_pos.groupby('prefecture')['latitude'].count().rename('prefecture_latitude_count'),
    ], axis=1)

    df_pos_city = pd.concat([
        df_pos.groupby('city_code')['latitude'].mean().rename('city_latitude'),
        df_pos.groupby('city_code')['longitude'].mean().rename('city_longitude'),
        df_pos.groupby('city_code')['latitude'].std().rename('city_latitude_std'),
        df_pos.groupby('city_code')['longitude'].std().rename('city_longitude_std'),
        df_pos.groupby('city_code')['latitude'].count().rename('city_latitude_count'),
    ], axis=1)

    df_pos_city2_choume = pd.concat([
        df_pos.groupby(['city_code', 'city2_choume'])['latitude'].mean().rename('city2_latitude'),
        df_pos.groupby(['city_code', 'city2_choume'])['longitude'].mean().rename('city2_longitude'),
    ], axis=1)

    df_pos_city2 = pd.concat([
        df_pos.groupby(['city_code', 'city2'])['latitude'].mean().rename('city2_latitude'),
        df_pos.groupby(['city_code', 'city2'])['longitude'].mean().rename('city2_longitude'),
    ], axis=1)

    df_pos_city2 = pd.concat([
        df_pos_city2_choume.reset_index().rename(columns={ 'city2_choume': 'city2' }).set_index(['city_code', 'city2']),
        df_pos_city2,
    ])
    df_pos_city2 = df_pos_city2.loc[~df_pos_city2.index.duplicated()]

    df = df.merge(df_pos_pref, how='left', on='prefecture')
    df = df.merge(df_pos_city, how='left', on='city_code')
    df = df.merge(df_pos_city2, how='left', on=['city_code', 'city2'])
    
    # df = df.set_index('index').loc[df_orig['index']]
    
    # return df.loc[df_orig.index]
    return df

display(add_pos(df, df_pos))


,type,zone_type,city_code,prefecture,city,city2,nearest_sta,nearest_sta_dist,price,tubo_price,...,prefecture_latitude_std,prefecture_longitude_std,prefecture_latitude_count,city_latitude,city_longitude,city_latitude_std,city_longitude_std,city_latitude_count,city2_latitude,city2_longitude
0,宅地(土地),住宅地,1101,北海道,札幌市中央区,南８条西,西１８丁目,15.0,50000000,500000.0,...,0.560714,1.173699,25980,43.053101,141.337146,0.012970,0.017902,990,NaN,NaN
1,宅地(土地と建物),住宅地,1102,北海道,札幌市北区,屯田８条,NaN,NaN,32000000,NaN,...,0.560714,1.173699,25980,43.114164,141.338905,0.026226,0.023171,1057,NaN,NaN
2,宅地(土地と建物),住宅地,1107,北海道,札幌市西区,八軒９条西,新川(北海道),16.0,37000000,NaN,...,0.560714,1.173699,25980,43.080747,141.289950,0.013338,0.019257,583,NaN,NaN
3,宅地(土地と建物),住宅地,1108,北海道,札幌市厚別区,厚別東５条,森林公園(北海道),5.0,35000000,NaN,...,0.560714,1.173699,25980,43.038951,141.475334,0.013322,0.012861,204,NaN,NaN
4,宅地(土地),宅地見込地,4105,宮城県,仙台市泉区,福岡,泉中央,120.0,380000,6400.0,...,0.270941,0.286563,4791,38.326343,140.861240,0.017573,0.038324,180,38.388372,140.733106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4880200,宅地(土地),住宅地,47362,沖縄県,島尻郡八重瀬町,字具志頭,NaN,NaN,500000,26000.0,...,0.414110,0.797726,1221,26.141621,127.726234,0.020329,0.015657,23,26.125995,127.747182
4880201,宅地(土地と建物),住宅地,47362,沖縄県,島尻郡八重瀬町,字小城,NaN,NaN,6000000,NaN,...,0.414110,0.797726,1221,26.141621,127.726234,0.020329,0.015657,23,26.157841,127.704558
4880202,宅地(土地と建物),住宅地,47362,沖縄県,島尻郡八重瀬町,字東風平,NaN,NaN,43000000,NaN,...,0.414110,0.797726,1221,26.141621,127.726234,0.020329,0.015657,23,26.152087,127.721199
4880203,農地,NaN,47362,沖縄県,島尻郡八重瀬町,字仲座,NaN,NaN,3600000,NaN,...,0.414110,0.797726,1221,26.141621,127.726234,0.020329,0.015657,23,26.111990,127.724532


In [3]:
# available features
features = [
    'type',
    # 'zone_type',
    'city_plan',
    'city_code',
    'prefecture',
    'city',
    'city2',
    'nearest_sta',
    'nearest_sta_dist',
    'age',
    # 'structure',
    'area',
    'floor_area',
    # 'layout', # many nan
    # 'front_road_dir',
    # 'front_road_type',
    'front_road_width',
    # 'building_cov',
    # 'floor_cov',
    'building_year',
    'time',
    # 'choume'
    # 'sawa',
    # 'numa',
    # 'tani',
    # 'ken',
    'city_code_low',
    # 'city_code_low2',
    # 'city2_last',
    # 'city2_first',
    # 'city_last',
    # 'city_first',
    # 'city2_len',
    # 'city_len',
    # 'floor_area_area',
    # 'area_front_road_width',
    # 'area_front_road_width2',
    'prefecture_latitude',
    'prefecture_longitude',
    'prefecture_latitude_std',
    'prefecture_longitude_std',
    'prefecture_latitude_count',
    'city_latitude',
    'city_longitude',
    'city_latitude_std',
    'city_longitude_std',
    'city_latitude_count',
    'city2_latitude',
    'city2_longitude',
    # 'x',
    # 'x2',
]

In [4]:
def show_importance(model, features):
    importances = model.feature_importances_
    feature_imp = pd.DataFrame(zip(importances, features), columns=['value', 'feature'])
    feature_imp = feature_imp.sort_values('value')
    display(feature_imp)

In [5]:
class RemlModel:
    def __init__(self, df_all):
        self._city_code_map = df_all.groupby(['prefecture', 'city'])[['city_code']].nth(0)
        self._prefecture_pos_map = df_all.groupby(['prefecture'])[[
            'prefecture_latitude', 
            'prefecture_longitude',
            'prefecture_latitude_std',
            'prefecture_longitude_std',
            'prefecture_latitude_count',
        ]].nth(0)
        self._city_pos_map = df_all.groupby(['city_code'])[[
            'city_latitude',
            'city_longitude',
            'city_latitude_std',
            'city_longitude_std',
            'city_latitude_count',
        ]].nth(0)
        self._city2_pos_map = df_all.groupby(['city_code', 'city2'])[[
            'city2_latitude',
            'city2_longitude',
        ]].nth(0)
        self._dtypes = df_all.dtypes

    def fit(self, df, mean_only=False):
        df = df.copy()
        
        cat_features = [
            'type',
            'city_plan',
            'prefecture',
            'city',
            'city2',
            'nearest_sta',
            # 'structure',
            # 'layout',
            # 'front_road_dir',
            # 'front_road_type',
            # 'city2_last',
            # 'city2_first',
            # 'city_last',
            # 'city_first',
        ]
        num_features = [x for x in features if x not in cat_features]
        # num_features = [features.index(x) for x in num_features]
        # cat_features = [features.index(x) for x in cat_features]
        
        def create_model(alpha=None):
            n_est_scale = 10
            
            model = lgb.LGBMRegressor(
                alpha=alpha,
                objective='regression' if alpha is None else 'quantile',
                n_estimators=100 * n_est_scale,
                learning_rate=0.1 / n_est_scale,
                # colsample_bytree=0.5,
                subsample=0.5,
                subsample_freq=1,
                # extra_trees=True,
                importance_type='gain',
                random_state=1, 
                n_jobs=-1
            )
            et = lgb.LGBMRegressor(
                n_estimators=100 * n_est_scale,
                learning_rate=0.1 / n_est_scale,
                colsample_bytree=0.5,
                subsample=0.5,
                subsample_freq=1,
                extra_trees=True,
                random_state=1, 
                n_jobs=-1
            )
            rf = lgb.LGBMRegressor(
                n_estimators=100 * n_est_scale,
                learning_rate=0.1 / n_est_scale,
                boosting_type='rf',
                colsample_bytree=0.5,
                subsample=0.5,
                subsample_freq=1,
                # extra_trees=True,
                random_state=1, 
                n_jobs=-1
            )
            
            xg = xgb.XGBRegressor(
                random_state=1,
                n_jobs=-1,
            )
            cb = CatBoostRegressor(
                verbose=0,
                random_state=1,
                thread_count=-1,
            )
            
            num_transformer = Pipeline([
                ("imputer", SimpleImputer(strategy="median")), 
                # ("scaler", StandardScaler()),
                ("qt", QuantileTransformer(output_distribution='normal', random_state=1)),
                ("fe", FeatureUnion([
                    ('pt', 'passthrough'),
                    ('pca', PCA(n_components=2)),
                ])),
            ])

            cat_transformer = Pipeline([
                ('fe',  FeatureUnion([
                    ("ce", ce.count.CountEncoder(min_group_size=1)), # good
                    # ("te", ce.target_encoder.TargetEncoder()), # bad
                    # ("le", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=np.nan)),
                ])),
                # ("le", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=np.nan)),
            ], memory=Memory('/tmp/joblib'))
            
            ridge = Pipeline([
                ('ct', ColumnTransformer([
                    ("num", num_transformer, np.arange(len(num_features))),
                ])),
                ('model', RidgeCV()),
            ])
            
            # model = RFE(model)
            # model = BaggingRegressor(model, n_estimators=10, random_state=1)
            # model = StackingRegressor([('m', model)], final_estimator=model, cv=None, n_jobs=None, passthrough=True, verbose=0) # good
            model = StackingRegressor([
                ('m', model),
                ('qt_m', TransformedTargetRegressor(model, transformer=QuantileTransformer(output_distribution='normal', random_state=1))),
                ('et', et),
                ('rf', rf),
                ('xg', xg), # good
                ('cb', cb), # good
                ('ridge', ridge),
            ], final_estimator=model, cv=None, n_jobs=None, passthrough=True, verbose=0) # good
            # model = StackingRegressor([
            #     ('m', StackingRegressor([('m', model)], final_estimator=model, cv=None, n_jobs=None, passthrough=True, verbose=0))
            # ], final_estimator=model, cv=None, n_jobs=None, passthrough=True, verbose=0) # not good

            model = Pipeline([
                ('ct', ColumnTransformer([
                    ("num", 'passthrough', num_features),
                    # ("num", num_transformer, num_features),
                    ("cat", cat_transformer, cat_features),
                ])),
                ('model', model),
            ])

            # model = BaggingRegressor(model, n_estimators=10, random_state=1)
            
            return model
        
        df = df.sample(frac=1, random_state=1) # for ordinal encoder
        
        # y = np.log(df['price'])
        y = np.log(df['price'] / df['area'])
        # plt.hist(y, bins=100)
        # plt.show()
        X = self.calc_features(df)
        
        model = create_model()
        model.fit(X, y)
        
        # show_importance(model.named_steps['model'], num_features + cat_features)
        # show_importance(model.named_steps['model'].estimators_[0], num_features + cat_features)
        
        self._model = model
        
        self.mean_only = mean_only
        if not mean_only:
            model = create_model(alpha=0.1)
            model.fit(X, y)
            self._model_10 = model
            model = create_model(alpha=0.9)
            model.fit(X, y)
            self._model_90 = model
    
    def predict(self, df):
        df = df.copy()
        
        for col in df.columns:
            if col in self._dtypes:
                df[col] = df[col].astype(self._dtypes[col])
        
        X = self.calc_features(df)
        # return np.exp(self._model.predict(X))
        
        df_res = pd.DataFrame(index=df.index)
        df_res['price'] = np.exp(self._model.predict(X)) * df['area'].values
        if not self.mean_only:
            df_res['price_10'] = np.exp(self._model_10.predict(X)) * df['area'].values
            df_res['price_90'] = np.exp(self._model_90.predict(X)) * df['area'].values
        # df_res['price'] = np.exp(self._model.predict(X))
        # df_res['price_10'] = np.exp(self._model_10.predict(X))
        # df_res['price_90'] = np.exp(self._model_90.predict(X))
        
        return df_res
    
    def metadata(self):
        return {
            'unique_values': self._unique_values,
        }
    
    def calc_features(self, df):
        df_orig = df.reset_index()
        df = df_orig.copy()
        
        df = df.merge(self._city_code_map, how='left', on=['prefecture', 'city'])
        df = df.merge(self._prefecture_pos_map, how='left', on=['prefecture'])
        df = df.merge(self._city_pos_map, how='left', on=['city_code'])
        df = df.merge(self._city2_pos_map, how='left', on=['city_code', 'city2'])
        df = df.set_index('index').loc[df_orig['index']]
        
#         from kanjize import kanji2number
#         def conv_kanji(x):
#             if isinstance(x, float):
#                 return 0
#             print(x)
#             return kanji2number(x)
        
#         df['choume'] = df['city2'].str.extract(r'([一二三四五六七八九十]+)丁目')
#         df['choume'] = df['choume'].apply(conv_kanji).astype('int')
        
        # df['sawa'] = df['city2'].str.contains('沢')
        # df['numa'] = df['city2'].str.contains('沼')
        # df['tani'] = df['city2'].str.contains('谷')

        # df['ken'] = df['prefecture'].str.contains('県')
        df['city_code_low'] = df['city_code'] % 1000
        # df['city_code_low2'] = df['city_code'] % 100
    
        df['age'] = df['time'] - df['building_year'] * 100
        
        df['city2_last'] = df['city2'].str[-1:]
        df['city2_first'] = df['city2'].str[:1]
        df['city_last'] = df['city'].str[-1:]
        df['city_first'] = df['city'].str[:1]
        df['city2_len'] = df['city2'].str.len()
        df['city_len'] = df['city'].str.len()
        
        df['floor_area_area'] = df['floor_area'] / df['area']
        df['area_front_road_width'] = df['area'] / df['front_road_width']
        df['area_front_road_width2'] = df['area'] / df['front_road_width'] ** 2
        
        df['x'] = df['city_latitude_std'] * df['city_longitude_std'] / df['city_latitude_count']
        df['x2'] = df['area'] / df['x']

        # display(df)
        # display(df_orig)
        
        df = df[features]

        return df

In [ ]:
df = pd.read_pickle('/tmp/all.pkl')
df_pos = pd.read_pickle('/tmp/pos_all.pkl')
df = add_pos(df, df_pos)

# df = df.loc[df['type'] == '宅地(土地)']
# df = df.loc[df['type'] == '宅地(土地と建物)']
# df = df.loc[df['type'] == '中古マンション等']
# df = df.loc[df['type'] == '農地']
# df = df.loc[df['type'] == '林地']
# df = df.loc[df['type'].isin(['宅地(土地と建物)', '中古マンション等'])]
df = df.loc[df['type'].isin(['宅地(土地)', '宅地(土地と建物)', '中古マンション等'])]

model = RemlModel(df)

# df = df.iloc[::100]
df = df.drop(columns='city_code')
df = df.drop(columns=[x for x in df.columns if 'latitude' in x or 'longitude' in x])

model.fit(df)
# model.fit(df, mean_only=True)
# print(model.metadata())

data = cloudpickle.dumps(model)
data = lzma.compress(data)
model_name = '20230222_pos'
with open('/home/jovyan/data/{}.xz'.format(model_name), 'wb') as f:
    f.write(data)
print('model size {}'.format(os.path.getsize('/home/jovyan/data/{}.xz'.format(model_name))))

cv = KFold(shuffle=True, random_state=1)

def calc_score(y, y_pred):
    return r2_score(np.log(y), np.log(y_pred))
    # return pearsonr(np.log(y), np.log(y_pred))[0]

scores = []
for train_idx, val_idx in cv.split(df):
    model.fit(df.iloc[train_idx], mean_only=True)
    
    df_val = df.iloc[val_idx]
    df_res = model.predict(df_val)
    y_pred = df_res['price']
    y = df_val['price']
    score = calc_score(y, y_pred)
    scores += [{ 'score': score, 'type': 'all' }]
    print('score {}'.format(score))
    
    for tp, df_type in df_val.groupby('type'):
        score = calc_score(y.loc[df_type.index], y_pred.loc[df_type.index])
        scores += [{ 'score': score, 'type': tp }]
        print('{} score {}'.format(tp, score))
        
    for tp, df_type in df_val.loc[df_val['prefecture'] == '東京都'].groupby('type'):
        score = calc_score(y.loc[df_type.index], y_pred.loc[df_type.index])
        scores += [{ 'score': score, 'type': tp + '_東京都' }]
        print('{}_東京都 score {}'.format(tp, score))

df_score = pd.DataFrame(scores)
display(df_score.groupby('type')[['score']].describe())

model size 17448552


In [ ]:
def create_metadata(df):
    unique_values = {}
    for col in df.columns:
        unique_values[col] = df[col].sort_values().unique().tolist()

    x = df['prefecture']
    x = x + ':' + df['city']
    x = x + ':' + df['city2']
    x = x + ':' + df['nearest_sta']
    unique_values['prefecture_city_city2_nearest_sta'] = x.sort_values().unique().tolist()
    
    return {
        'unique_values': unique_values,
    }

df = pd.read_pickle('/tmp/all.pkl')
df = df.loc[df['type'].isin(['宅地(土地)', '宅地(土地と建物)', '中古マンション等'])]
metadata = create_metadata(df)

data = cloudpickle.dumps(metadata)
data = lzma.compress(data)
with open('/home/jovyan/data/{}_metadata.xz'.format(model_name), 'wb') as f:
    f.write(data)